In [5]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Predefined list of stocks
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'JNJ', 'XOM', 'JPM', 'PG', 'NVDA']  # Broad market overview

# Define the time period
start_date = '2019-01-01'
end_date = '2024-01-01'

# dashborad
# Fetch and Process data
def fetch_data(ticker, start_date, end_date):
  ticker = yf.Ticker(ticker)
  hist = ticker.history(start=start_date, end=end_date)
  if hist.empty:
        raise ValueError(f"No data found for {ticker} in the date range {start_date} to {end_date}")
  hist['MA50'] = hist['Close'].rolling(window=50).mean()
  hist['MA200'] = hist['Close'].rolling(window=200).mean()
  hist['Vatility'] = hist['Close'].rolling(window=20).std()
  hist.fillna(method='ffill',inplace=True)
  return hist

# Intialize the app
app = dash.Dash(__name__)
app.title = 'Stock Dashboard'

# Define layout
app.layout = html.Div([
    html.H1('Stock Dashboard', style= {'textAlign': 'center'}),

    # Dropdown for selecting stock ticker
    html.Div([
        html.Label('Select Stock Ticker:'),
        dcc.Dropdown(
            id='ticker-dropdown',
            options=[{'label': ticker, 'value': ticker} for ticker in stocks],
            value=stocks[0]
        ),
    ], style={'width': '50%', 'margin': 'auto'}),

    # Date picker
    html.Div([
        html.Label('Select Date Range:'),
        dcc.DatePickerRange(
            id='date-picker',
            start_date= start_date,
            end_date= end_date,
            display_format='YYYY-MM-DD'
        ),
    ], style={'width': '50%', 'margin': 'auto'}),
   # Place holder for graphs
    html.Div([
        dcc.Graph(id='stock-price-graph'),
        dcc.Graph(id='stock-volumn-graph'),
        dcc.Graph(id='volatility-graph'),
    ])
])

# Define callback
@app.callback(
    [Output('stock-price-graph', 'figure'),
     Output('stock-volumn-graph', 'figure'),
     Output('volatility-graph', 'figure')],
    [Input('ticker-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)

def update_graphs(ticker, start_date, end_date):
  # fetch data
  hist = fetch_data(ticker, start_date, end_date)
  hist.fillna(method='ffill', inplace=True)

  # Create price and MA chart
  price_fig = go.Figure()
  price_fig.add_trace(go.Scatter(x=hist.index, y=hist['Close'], mode='lines', name='Close'))
  price_fig.add_trace(go.Scatter(x=hist.index, y=hist['MA50'], mode='lines', name='MA50'))
  price_fig.add_trace(go.Scatter(x=hist.index, y=hist['MA200'], mode='lines', name='MA20'))
  price_fig.update_layout(title=f'{ticker} Stock Prices And Moving Average', xaxis_title='Date', yaxis_title='Price')

  # Create volume histogram
  volume_fig = go.Figure()
  volume_fig.add_trace(go.Bar(x=hist.index, y=hist['Volume'], name='Volume'))
  volume_fig.update_layout(title=f'{ticker} Volume Distribution', xaxis_title='Volume')

  # Create volatility scatter plot
  volatility_fig = go.Figure()
  volatility_fig.add_trace(go.Scatter(x=hist.index, y=hist['Vatility'], mode='markers', name='Volatility'))
  volatility_fig.update_layout(title=f'{ticker} Volatility Assessment', xaxis_title='Date', yaxis_title='Volatility')

  return price_fig, volume_fig, volatility_fig

if __name__ == '__main__':
  app.run_server(debug=True)

<IPython.core.display.Javascript object>